This notebook will be used to process Sentinel-1 L2A products from GEE. These products have RGB, NIR, and SWIR1, as well as NDVI, NDWI, MSAVI2, and BSI spectral indices. The plan is to delineate and classify water as 0 and land as 1 using NDVI and NDWI values. NDVI > 0.0 and NDWI < 0.0 will result in a pixel being labeled 1 for land, and vice versa for water.

Assuming the images have been downloaded from GEE Python API (check notebook "sentinel_one_two.ipynb" under the "gee_python" folder).

1. Load in the .tif files and the needed bands using gdal or similar
2. Label the RGB imagery using NDVI and NDWI thresholding as water or land
3. Save the classified image for labeling Sentinel-1 imagery

# Import depenedencies

In [10]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import rasterio
from rasterio.transform import from_origin
import numpy as np
import matplotlib.ticker as mticker
import os
from skimage.filters import threshold_otsu
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import cv2
from osgeo import gdal, osr
from datetime import datetime

# Functions

In [349]:
# functions
def map_indices(imagepath):
    with rasterio.open(imagepath) as ind_src:
        ndvi = ind_src.read(1)  # Read first band (assume NDVI is in first band)
        ndwi = ind_src.read(2)  # Read first band (assume NDVI is in first band)
        msavi2 = ind_src.read(3)  # Read first band (assume NDVI is in first band)
        bsi = ind_src.read(4)  # Read first band (assume NDVI is in first band)
    
    
    # Visualizing each band using matplotlib
    plt.figure(figsize=(10, 10))

    # NDVI visualization
    plt.subplot(2, 2, 1)
    plt.imshow(ndvi, cmap='RdYlGn')  # 'RdYlGn' is a color map for NDVI-like data
    plt.colorbar(label='NDVI')
    plt.title('NDVI')

    # NDWI visualization
    plt.subplot(2, 2, 2)
    plt.imshow(ndwi, cmap='Blues')  # 'Blues' is good for water index
    plt.colorbar(label='NDWI')
    plt.title('NDWI')

    # MSAVI2 visualization
    plt.subplot(2, 2, 3)
    plt.imshow(msavi2, cmap='Greens')  # 'Greens' for vegetation index
    plt.colorbar(label='MSAVI2')
    plt.title('MSAVI2')

    # BSI visualization
    plt.subplot(2, 2, 4)
    plt.imshow(bsi, cmap='BrBG')  # 'BrBG' for contrast between soil and water
    plt.colorbar(label='BSI')
    plt.title('BSI')

    plt.tight_layout()
    plt.show()

    return ndvi, ndwi, msavi2, bsi

def plot_class(man_class, otsu_class, image_path):
    # Open the image to get the coordinates and transform
    with rasterio.open(image_path) as src:
        transform = src.transform
        height, width = man_class.shape
        top_left = rasterio.transform.xy(transform, 0, 0, offset='center')
        bottom_right = rasterio.transform.xy(transform, height-1, width-1, offset='center')

    min_easting, max_northing = top_left
    max_easting, min_northing = bottom_right

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Manual classification visualization
    ax[0].imshow(man_class, cmap='coolwarm', extent=[min_easting, max_easting, min_northing, max_northing])
    ax[0].set_title(f'{image_path[-14:-4]} (Manual)')
    ax[0].set_xlabel('Easting (meters)')
    ax[0].set_ylabel('Northing (meters)')
    ax[0].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # Otsu classification visualization
    ax[1].imshow(otsu_class, cmap='coolwarm', extent=[min_easting, max_easting, min_northing, max_northing])
    ax[1].set_title(f'{image_path[-14:-4]} (Otsu)')
    ax[1].set_xlabel('Easting (meters)')
    ax[1].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # Add a custom legend for the colors
    red_patch = mpatches.Patch(color='red', label='Land')
    blue_patch = mpatches.Patch(color='blue', label='Water')
    plt.legend(handles=[red_patch, blue_patch], loc='lower right', title="Classification")

    # Show the plot with layout adjustments
    plt.tight_layout()
    plt.show()

def classify_images(imagepath):
    with rasterio.open(imagepath) as ind_src:
        ndvi = ind_src.read(1)  # Read first band (assume NDVI is in first band)
        ndwi = ind_src.read(2)  # Read first band (assume NDVI is in first band)
        msavi2 = ind_src.read(3)  # Read first band (assume NDVI is in first band)
        bsi = ind_src.read(4)  # Read first band (assume NDVI is in first band)
    
    # # Alternatively, mask NaN values
    ndvi_clean = ndvi[~np.isnan(ndvi)]
    ndwi_clean = ndwi[~np.isnan(ndwi)]

    # Calculate Otsu thresholds for NDVI and NDWI
    otsu_ndvi_threshold = threshold_otsu(ndvi_clean)
    otsu_ndwi_threshold = threshold_otsu(ndwi_clean)

    # Create a mask for land and water
    # Land: NDVI > 0.0 and NDWI < 0.0
    land_mask = (ndvi > 0.0) & (ndwi < 0.0)
    otsu_land_mask = (ndvi > otsu_ndvi_threshold) & (ndwi < otsu_ndwi_threshold)


    # Water: NDVI <= 0.0 or NDWI >= 0.0 (inverse of land)
    water_mask = ~land_mask  # Invert the land mask to get water
    otsu_water_mask = ~otsu_land_mask  # Invert the land mask to get water


    # Combine masks: 1 for land, 0 for water
    man_classified = np.zeros_like(ndvi, dtype=np.uint8)
    man_classified[land_mask] = 1  # Mark land as 1
    man_classified[water_mask] = 0  # Mark water as 0


    # Combine masks: 1 for land, 0 for water
    otsu_classified = np.zeros_like(ndvi, dtype=np.uint8)
    otsu_classified[otsu_land_mask] = 1  # Mark land as 1
    otsu_classified[otsu_water_mask] = 0  # Mark water as 0

    return man_classified, otsu_classified

def normalize_rgb(rgb):
    """Normalize the RGB values to the range [0, 255]."""
    rgb_min, rgb_max = np.min(rgb), np.max(rgb)
    rgb_normalized = (rgb - rgb_min) / (rgb_max - rgb_min)  # Normalize to [0, 1]
    rgb_normalized *= 255  # Scale to [0, 255]
    return rgb_normalized.astype(np.uint8)  # Convert to uint8 for display

def plot_class_with_rgb(rgb_path, man_class, otsu_class, kmeans_class, gmm_class):
    # Open the RGB image
    with rasterio.open(rgb_path) as rgb_src:
        rgb = rgb_src.read([3, 2, 1])  # Read the first three bands (R, G, B)
        rgb = np.moveaxis(rgb, 0, -1)  # Rearrange dimensions to (height, width, bands)
        rgb = normalize_rgb(rgb)  # Normalize the RGB image to [0, 255]

        transform = rgb_src.transform
        height, width = rgb.shape[:2]
        top_left = rasterio.transform.xy(transform, 0, 0, offset='center')
        bottom_right = rasterio.transform.xy(transform, height-1, width-1, offset='center')

    # Extract easting and northing from the corners
    min_easting, max_northing = top_left
    max_easting, min_northing = bottom_right

    fig, ax = plt.subplots(1, 5, figsize=(30, 6))  # 5 subplots for RGB, manual, Otsu, K-Means, and GMM

    # RGB image visualization
    ax[0].imshow(rgb, extent=[min_easting, max_easting, min_northing, max_northing])
    ax[0].set_title('RGB Image')
    ax[0].set_xlabel('Easting (meters)')
    ax[0].set_ylabel('Northing (meters)')
    ax[0].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # Custom legends for classification plots
    red_patch = mpatches.Patch(color='red', label='Land')
    blue_patch = mpatches.Patch(color='blue', label='Water')

    # Manual classification visualization
    ax[1].imshow(man_class, cmap='coolwarm', extent=[min_easting, max_easting, min_northing, max_northing])
    ax[1].set_title(f'{rgb_path[-14:4]} (Manual)')
    ax[1].set_xlabel('Easting (meters)')
    ax[1].legend(handles=[red_patch, blue_patch], loc='lower right', title="Classification")
    ax[1].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # Otsu classification visualization
    ax[2].imshow(otsu_class, cmap='coolwarm', extent=[min_easting, max_easting, min_northing, max_northing])
    ax[2].set_title(f'{rgb_path[-14:-4]} (Otsu)')
    ax[2].set_xlabel('Easting (meters)')
    ax[2].legend(handles=[red_patch, blue_patch], loc='lower right', title="Classification")
    ax[2].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # K-Means classification visualization
    ax[3].imshow(kmeans_class, cmap='coolwarm_r', extent=[min_easting, max_easting, min_northing, max_northing])
    ax[3].set_title(f'{rgb_path[-14:-4]} (K-Means)')
    ax[3].set_xlabel('Easting (meters)')
    ax[3].legend(handles=[red_patch, blue_patch], loc='lower right', title="Classification")
    ax[3].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # GMM classification visualization
    ax[4].imshow(gmm_class, cmap='coolwarm_r', extent=[min_easting, max_easting, min_northing, max_northing])
    ax[4].set_title(f'{rgb_path[-14:-4]} (GMM)')
    ax[4].set_xlabel('Easting (meters)')
    ax[4].legend(handles=[red_patch, blue_patch], loc='lower right', title="Classification")
    ax[4].xaxis.set_major_locator(mticker.MaxNLocator(5))  # Reduce x-axis ticks

    # Show the plot with layout adjustments
    plt.tight_layout()
    plt.show()


def kmeans_land_water(image_path, n_clusters=2):
    with rasterio.open(image_path) as src:
        img = src.read()  # Read all bands
        height, width = img.shape[1], img.shape[2]
        
        # Reshape to (num_pixels, num_bands)
        img_flat = img.reshape((img.shape[0], height * width)).T

        # Apply K-Means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(img_flat)
        classified_img = kmeans.labels_.reshape((height, width))

    return classified_img

def gmm_land_water(image_path, n_components=2):
    with rasterio.open(image_path) as src:
        img = src.read()  # Read all bands
        height, width = img.shape[1], img.shape[2]
        
        # Reshape to (num_pixels, num_bands)
        img_flat = img.reshape((img.shape[0], height * width)).T

        # Apply Gaussian Mixture Model
        gmm = GaussianMixture(n_components=n_components, random_state=42).fit(img_flat)
        gmm_labels = gmm.predict(img_flat).reshape((height, width))

    return gmm_labels

def export_labels_to_tif(labels, reference_tif_path, output_tif_path):
    # Open the reference Sentinel-1 image to get the transform and CRS
    with rasterio.open(reference_tif_path) as src:
        transform = src.transform
        crs = src.crs
        height, width = labels.shape

        # Define metadata for the new GeoTIFF file
        meta = src.meta.copy()
        meta.update({
            'driver': 'GTiff',
            'height': height,
            'width': width,
            'count': 1,  # Single band (the labels)
            'dtype': 'uint8',  # Assuming the labels are 0 or 1
            'crs': crs,
            'transform': transform
        })

        # Write the labels array to a new GeoTIFF file
        with rasterio.open(output_tif_path, 'w', **meta) as dst:
            dst.write(labels.astype('uint8'), 1)  # Write the labels as the first band

def get_rgb_avg(rgb_avg_path, combined_ims):
    # Open all the combined .vrt files and read their bands
    all_bands = []
    
    # Loop over each file to read its bands
    for f in combined_ims:
        ds = gdal.Open(f)
        bands = [ds.GetRasterBand(i+1).ReadAsArray() for i in range(ds.RasterCount)]
        all_bands.append(bands)
    
    # Stack the bands across all images (axis=0 for stacking across different images)
    stacked_bands = [np.stack([image_bands[i] for image_bands in all_bands], axis=0) for i in range(len(all_bands[0]))]
    
    # Compute the mean for each band across the stacked images (axis=0 is across images)
    mean_bands = [np.mean(stacked_band, axis=0) for stacked_band in stacked_bands]

    # Create a new GeoTIFF with the averaged bands
    driver = gdal.GetDriverByName('GTiff')
    
    # Use the first file for spatial reference (CRS and geotransform)
    ds = gdal.Open(combined_ims[0])
    
    # Create an output file with the same dimensions and number of bands as the input
    result = driver.Create(rgb_avg_path, ds.RasterXSize, ds.RasterYSize, len(mean_bands), gdal.GDT_Float32)

    # Copy projection and geotransform from the original dataset
    result.SetProjection(ds.GetProjection())
    result.SetGeoTransform(ds.GetGeoTransform())

    # Write each averaged band to the output file
    for i, meanband in enumerate(mean_bands):
        result.GetRasterBand(i+1).WriteArray(meanband)

    # Close the result dataset to flush the data to disk
    result = None

    return rgb_avg_path

def perform_pca(image_path, output_pca_path):
    # Load the Sentinel-2 multi-band image using GDAL
    dataset = gdal.Open(image_path)

    # Read all bands as separate arrays
    bands = [dataset.GetRasterBand(i + 1).ReadAsArray() for i in range(dataset.RasterCount)]

    # Convert the list of bands into a 3D NumPy array (bands, rows, cols)
    bands_array = np.stack(bands, axis=0)

    # Reshape the bands array into (pixels, bands) for PCA
    pixels, bands_count = bands_array.shape[1] * bands_array.shape[2], bands_array.shape[0]
    flattened_image = bands_array.reshape(bands_count, -1).T  # Shape: (pixels, bands)

    # Convert to float32 for OpenCV PCA
    flattened_image = flattened_image.astype(np.float32)

    # Perform PCA using OpenCV (reduce to 1 principal component)
    mean, eigenvectors = cv2.PCACompute(flattened_image, mean=None, maxComponents=1)
    pca_result = cv2.PCAProject(flattened_image, mean, eigenvectors)

    # Reshape the PCA result back to the original image dimensions
    pca_image = pca_result.reshape(bands_array.shape[1], bands_array.shape[2])
    # pca_image = np.nan_to_num(pca_image, nan=0.0, posinf=255.0, neginf=0.0)

    # Normalize the PCA image to 0-255 for OpenCV processing
    pca_image_normalized = cv2.normalize(pca_image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Save the PCA-reduced image
    output = gdal.GetDriverByName('GTiff').Create(output_pca_path, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Float32)
    output.SetProjection(dataset.GetProjection())
    output.SetGeoTransform(dataset.GetGeoTransform())
    output.GetRasterBand(1).WriteArray(pca_image_normalized)
    output.FlushCache()  # Ensure data is written to disk
    output = None

def get_labels(labelpath):
    man_ims = [os.path.join(labelpath, f'manual/{file}') for file in os.listdir(os.path.join(labelpath, f'manual')) if file.endswith('.tif')]
    man_ims = sorted(man_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))
    
    otsu_ims = [os.path.join(labelpath, f'otsu/{file}') for file in os.listdir(os.path.join(labelpath, f'otsu')) if file.endswith('.tif')]
    otsu_ims = sorted(otsu_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))
    
    kmeans_ims = [os.path.join(labelpath, f'kmeans/{file}') for file in os.listdir(os.path.join(labelpath, f'kmeans')) if file.endswith('.tif')]
    kmeans_ims = sorted(kmeans_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))
    
    gmm_ims = [os.path.join(labelpath, f'gmm/{file}') for file in os.listdir(os.path.join(labelpath, f'gmm')) if file.endswith('.tif')]
    gmm_ims = sorted(gmm_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))

    return man_ims, otsu_ims, kmeans_ims, gmm_ims

def register_images(reference_image, target_image):
    # Define warp mode: use affine transformation (can also use cv2.MOTION_EUCLIDEAN)
    warp_mode = cv2.MOTION_TRANSLATION

    # Initialize the transformation matrix (2x3 affine transformation matrix)
    warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Define criteria for the ECC algorithm
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000, 1e-6)

    # Perform the ECC algorithm to find the transformation matrix
    try:
        cc, warp_matrix = cv2.findTransformECC(reference_image, target_image, warp_matrix, warp_mode, criteria)
    except cv2.error as e:
        print(f"Error in ECC: {e}")
        return None

    return warp_matrix

def apply_transformation_to_all_bands(target_bands, warp_matrix, image_shape, output_dtype=np.float32):
    transformed_bands = []
    
    for band in target_bands:
        # Apply the transformation to the band
        transformed_band = cv2.warpAffine(band.astype(np.float32), warp_matrix, (image_shape[1], image_shape[0]), 
                                          flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        
        # Handle NaN or infinite values by replacing them with valid values (e.g., 0)
        # transformed_band = np.nan_to_num(transformed_band, nan=0.0, posinf=0.0, neginf=0.0)
        
        # Convert to the desired output data type
        transformed_band = transformed_band.astype(output_dtype)
        
        transformed_bands.append(transformed_band)
    
    return transformed_bands

def save_multiband_image_as_tiff(output_path, transformed_bands, reference_dataset, gdal_dtype=gdal.GDT_Float32):
    # Create an output GeoTIFF file with the same dimensions and the same number of bands
    driver = gdal.GetDriverByName('GTiff')
    out_dataset = driver.Create(output_path, reference_dataset.RasterXSize, reference_dataset.RasterYSize, len(transformed_bands), gdal_dtype)

    # Set the projection and geotransform from the reference dataset
    out_dataset.SetProjection(reference_dataset.GetProjection())
    out_dataset.SetGeoTransform(reference_dataset.GetGeoTransform())

    # Write each transformed band to the output file
    for i, transformed_band in enumerate(transformed_bands):
        out_dataset.GetRasterBand(i + 1).WriteArray(transformed_band)

    # Flush data to disk
    out_dataset.FlushCache()
    out_dataset = None

def calculate_indices(impathlist):
    for im in impathlist:
        testds = gdal.Open(im)

        red = testds.GetRasterBand(3).ReadAsArray()  #.astype(float)
        green = testds.GetRasterBand(2).ReadAsArray()    #.astype(float)
        blue = testds.GetRasterBand(1).ReadAsArray() #.astype(float)
        nir = testds.GetRasterBand(4).ReadAsArray()  #.astype(float)
        swir1_10m = testds.GetRasterBand(5).ReadAsArray()    #.astype(float)


        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (nir - red) / (nir + red)
            ndvi[ndvi == np.inf] = np.nan


        with np.errstate(divide='ignore', invalid='ignore'):
            ndwi = (green - nir) / (green + nir)
            ndwi[ndwi == np.inf] = np.nan

        with np.errstate(divide='ignore', invalid='ignore'):
            msavi2 = (2 * nir + 1 - np.sqrt((2 * nir + 1) ** 2 - 8 * (nir - red))) / 2
            msavi2[msavi2 == np.inf] = np.nan

        with np.errstate(divide='ignore', invalid='ignore'):
            bsi = ((swir1_10m + red) - (nir + blue)) / ((swir1_10m + red) + (nir + blue))
            bsi[bsi == np.inf] = np.nan

        save_multiband_image_as_tiff(f'/home/wcc/Desktop/SabineRS/MSI/Indices/{im[-14:]}', [ndvi, ndwi, msavi2, bsi], testds)

# Import Imagery

In [322]:
# set the directory for where your images are located
# rgbpath ='/mnt/d/SabineMSI/RGB_NIR_SWIR1'
# indpath ='/mnt/d/SabineMSI/Indices'

rgbpath ='/home/wcc/Desktop/SabineRS/MSI/RGB_NIR_SWIR1/original'

rgb_ims = [os.path.join(rgbpath, file) for file in os.listdir(rgbpath) if file.endswith('.tif')]
rgb_ims = sorted(rgb_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))

# Register all rgb and indices images together to reduce impact of S2 perpendicular baseline changes
- https://www.geeksforgeeks.org/image-registration-using-opencv-python/
- https://medium.com/sentinel-hub/how-to-co-register-temporal-stacks-of-satellite-images-5167713b3e0b

In [330]:
rgb_avg_im = get_rgb_avg('/home/wcc/Desktop/SabineRS/MSI/RGB_NIR_SWIR1/rgb_average.tif', rgb_ims)
# PCA for the averaged image
perform_pca(rgb_avg_im, rgb_avg_im.replace('.tif', '_pca.tif'))

# PCA for each image
for im in rgb_ims:
    perform_pca(im,os.path.join(im[:-30], f'pca/{im[-21:-4]}_pca.tif'))


avg_pca_im = '/home/wcc/Desktop/SabineRS/MSI/RGB_NIR_SWIR1/rgb_average_pca.tif'
pcapath = os.path.join(rgbpath[:-9], f'pca')
pca_ims = [os.path.join(pcapath, file) for file in os.listdir(pcapath) if file.endswith('.tif')]
pca_ims = sorted(pca_ims, key=lambda x: datetime.strptime(x[-18:-8], '%Y-%m-%d'))

In [331]:
ref_ds = gdal.Open(avg_pca_im)
ref_im = ref_ds.GetRasterBand(1).ReadAsArray()

for i, im in enumerate(pca_ims):
    pca_ds = gdal.Open(im)
    pca_im = pca_ds.GetRasterBand(1).ReadAsArray() 

    warp_matrix = register_images(ref_im,pca_im)

    if warp_matrix is not None:
        reflectance_dataset = gdal.Open(rgb_ims[i])
        reflectance_bands = [reflectance_dataset.GetRasterBand(j + 1).ReadAsArray() for j in range(reflectance_dataset.RasterCount)]

        transformed_reflectance = apply_transformation_to_all_bands(reflectance_bands, warp_matrix, ref_im.shape)
        save_multiband_image_as_tiff(f'/home/wcc/Desktop/SabineRS/MSI/RGB_NIR_SWIR1/registered/{rgb_ims[i][-14:]}', transformed_reflectance, reflectance_dataset)

# Calculate NDVI, NDWI, MSAVI2, and BSI

In [333]:
regrgbpath = '/home/wcc/Desktop/SabineRS/MSI/RGB_NIR_SWIR1/registered'

reg_rgb_ims = [os.path.join(regrgbpath, file) for file in os.listdir(regrgbpath) if file.endswith('.tif')]
reg_rgb_ims = sorted(reg_rgb_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))

calculate_indices(reg_rgb_ims)

# Label data classes as 1 = Subaearial Land, or 0 = Water/subaqeuous land 

In [350]:
regindpath = '/home/wcc/Desktop/SabineRS/MSI/Indices'

reg_ind_ims = [os.path.join(regindpath, file) for file in os.listdir(regindpath) if file.endswith('.tif')]
reg_ind_ims = sorted(reg_ind_ims, key=lambda x: datetime.strptime(x[-14:-4], '%Y-%m-%d'))

In [352]:
# look at including the NDWI and NDVI in the kmeans and gmm models
# as of now it is only RGB, NIR and SWIR1

manclasses = []
otsuclasses = []
kmeansclasses = []
gmmclasses = []

for i, im in enumerate(reg_ind_ims):
    man_classified, otsu_classified = classify_images(im)
    km = kmeans_land_water(rgb_ims[i])
    gmm = gmm_land_water(rgb_ims[i])
    kmeansclasses.append(km)
    gmmclasses.append(gmm)
    manclasses.append(man_classified)
    otsuclasses.append(otsu_classified)

In [ ]:
ndvi, ndwi, msavi2, bsi = map_indices(reg_ind_ims[0])

# from some results, it looks like utilizing the otsu threshold is much 
# more accurate at delineating subaqueous and subaerial land
# a good example is reg_ind_ims[19] thru reg_ind_ims[26]

# classifies land and water
man_classified, otsu_classified = classify_images(reg_ind_ims[0])


plot_class_with_rgb(
    rgb_ims[0], 
    manclasses[0], 
    otsuclasses[0], 
    kmeansclasses[0], 
    gmmclasses[0]
    )

plot_class_with_rgb(
    rgb_ims[23], 
    manclasses[23], 
    otsuclasses[23], 
    kmeansclasses[23], 
    gmmclasses[23]
    )

# Export Labeled Sentinel-2 labeled data

In [303]:
for j, classes in enumerate([manclasses, otsuclasses, kmeansclasses, gmmclasses]):
    methods = ['manual', 'otsu', 'kmeans', 'gmm']
    for i, labels in enumerate(classes):
        reference_tif = rgb_ims[i]  # Path to the original Sentinel-1 image
        outdir = os.path.join(reference_tif[:16], f's2classifications/{methods[j]}')  # Define the directory path
        os.makedirs(outdir, exist_ok=True)  # Ensure the directory exists
        output_tif = os.path.join(outdir, f'{methods[j]}{reference_tif[-14:]}')  # Define the output file name
        export_labels_to_tif(labels, reference_tif, output_tif)